In [32]:
# import libraries
import pandas as pd

In [33]:
# import and merge data sets
jeopardy1 = pd.read_csv("jeopardy_1.csv")
jeopardy2 = pd.read_csv("jeopardy_2.csv")
df = pd.concat([jeopardy1, jeopardy2], axis=0)
#print(df.head(5))
#print(df.info())

In [34]:
# print column names
#print(df.columns) ## names for all but first column have leading space

# clean column names
# rename columns to remove leading space
df = df.rename(columns={'Show Number':'Show Number', 
                        ' Air Date':'Air Date', 
                        ' Round':'Round', 
                        ' Category':'Category', 
                        ' Value':'Value', 
                        ' Question':'Question', 
                        ' Answer':'Answer'}) ## safer way to rename
print(df.columns) ## check columns

Index(['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question',
       'Answer'],
      dtype='object')


In [35]:
# create function to search question column for specific cues and return a data frame containing only these questions

# view types of questions to inform what form search cues should take
#print(df.Question)

# filter for specific words
def word_find(data_frame, words):
    filter = lambda x: all(word.lower() in x.lower() for word in words)
    return data_frame.loc[data_frame.Question.apply(filter)]

# example of function use
filtered = word_find(df, ["king", "Henry VIII"])

#print(filtered.Question)
#print(len(filtered))

In [40]:
# modify data in a column to make it easier to analyze

# first explore the data in a column to see if special characters exist that will make analysis difficult
#print(df.Value)

# remove unwanted characters from series in column
df['Float_Value'] = df.Value.replace('[\$]', '', regex=True)
df['Float_Value'] = df.Float_Value.replace('[\,]', '', regex=True)
#print(len(df.Value))
#print(df.Float_Value) ## check that replace worked

# replace na values (here = none) with 0
df['Float_Value'] = df.Float_Value.replace('[None]', '0', regex=True)
#print(df.Float_Value) ## check that replace worked

# convert values in column to int/float
df['Float_Value'] = pd.to_numeric(df.Float_Value)
#print(df.dtypes) ## conversion worked
#print(df['Float_Value'].mean()) ## check mean

# this can all be done in 1 line of code with the following:
#jeopardy_data["Float Value"] = jeopardy_data["Value"].apply(lambda x: float(x[1:].replace(',','')) if x != "None" else 0)

In [42]:
# determine average value of question containing certain cues

# find rows with 'king'
filtered = word_find(df, ['king'])

# print mean value of questions containing 'king'
#print(filtered['Float_Value'].mean())
#output: 771.88

In [45]:
# determine number of times a particular answer is observed based on the cue used above to filter the data set

# create function to determine number of unique answer scores
def n_unique(data):
    return data['Answer'].value_counts()

# print number of unique answer scores associated with a filtered word
#print(n_unique(filtered))
#example: 'Henry VIII' is the answer to 55 questions containing the search cue 'king'

In [52]:
# identify the number of questions in the data set meeting specific criteria

# extract year from 'Air Date' column and add to 'Year' column
df['Year']=df['Air Date'].apply(lambda x: x.split('/')[2])
#print(df.Year)

# How many questions in the 1990s contained the word 'computer'?
filtered_data_90s = df[df['Year'].str.startswith('199') & df['Question'].str.contains('computer')]
#print(len(filtered_data_90s))
#output: 90

# How many questions in the 2000s contained the word 'computer'?
filtered_data_00s = df[df['Year'].str.startswith('200') & df['Question'].str.contains('computer')]
#print(len(filtered_data_00s))
#output: 244

In [54]:
# compare rounds and categories by counting the number of questions of each catregory based on round
round_cat = df.groupby(['Round', 'Category']).Question.count().reset_index()
#print(round_cat)

# calculate the number of questions in 'literature' category based on round and return round with highest count
maxcount = round_cat[round_cat.Category == 'LITERATURE'].Question.max()
#print(maxcount)
#output: 381

# print the round (and count) countaining the greatest number of questions in the 'literature' category
#print(round_cat[round_cat.Question == maxcount])